# Faster autoscaling on Amazon SageMaker realtime endpoints (Application Autoscaling)

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)


---

In this notebook we show how the new faster autoscaling feature helps scale sagemaker inference endpoints by almost 6x faster than earlier.

We deploy Meta's `Llama3-8B-Instruct` model to an Amazon SageMaker realtime endpoint using Text Generation Inference (TGI) Deep Learning Container (DLC) and apply <span style='color:green'><b>Application Autoscaling</b></span> scaling policies to the endpoint.


<div class="alert alert-block alert-warning">
    Please select <b>m5.2xlarge</b> or larger instance types when running this on Amazon SageMaker Notebook Instance.<br/>
    Select <b>conda_pytorch_p310</b> kernel when running this notebook on Amazon SageMaker Notebook Instance. <br/><br/>
    Ensure python version for the kernel is <b>3.10.x</b> (3.11 is not supported). <br/>
</div>

---

## Prerequisites



<div style="border: 1px solid #f00; border-radius: 5px; padding: 10px; background-color: #fee;">
Before using this notebook please ensure you have access to an active access token from HuggingFace and have accepted the license agreement from Meta.

- **Step 1:** Create user access token in HuggingFace (HF). Refer [here](https://huggingface.co/docs/hub/security-tokens) on how to create HF tokens.
- **Step 2:** Login to [HuggingFace](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/tree/main) and navigate to *Meta-Llama-3-8B-Instruct** home page.
- **Step 3:** Accept META LLAMA 3 COMMUNITY LICENSE AGREEMENT by following the instructions [here](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/tree/main)
- **Step 4:** Wait for the approval email from META (Approval may take any where b/w 1-3 hrs)
</div>

Install packages using uv, an extremely fast python package installer\
Read more about uv here <https://astral.sh/blog/uv>

In [1]:
# ensure python version of the selected kernel is not greater than 3.10
!python --version

Python 3.10.14


In [2]:
!pip install uv && uv pip install -U ipywidgets
!uv pip install -r requirements.txt

  Using cached uv-0.2.30-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
Using cached uv-0.2.30-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.9 MB)
Resolved 22 packages in 30ms                                         
Uninstalled 13 packages in 102ms
Installed 13 packages in 30ms.0.11                          
 - exceptiongroup==1.2.0 (from file:///home/conda/feedstock_root/build_artifacts/exceptiongroup_1704921103267/work)
 + exceptiongroup==1.2.2
 - ipython==8.22.2 (from file:///home/conda/feedstock_root/build_artifacts/ipython_1709559745751/work)
 + ipython==8.26.0
 - ipywidgets==8.1.2 (from file:///home/conda/feedstock_root/build_artifacts/ipywidgets_1707427226251/work)
 + ipywidgets==8.1.3
 - jupyterlab-widgets==3.0.10 (from file:///home/conda/feedstock_root/build_artifacts/jupyterlab_widgets_1707421892171/work)
 + jupyterlab-widgets==3.0.11
 - matplotlib-inline==0.1.6 (from file:///home/conda/feedstock_root/build_artifacts/matplotlib-inline_

In [3]:
# restart kernel
from IPython.core.display import HTML

HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
# load rich extension
%load_ext rich

In [5]:
import glob
import json
import os
import subprocess
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from getpass import getpass
from pathlib import Path
from statistics import mean
from uuid import uuid4

import boto3
import botocore
import sagemaker
from rich import box, print
from rich.console import Console
from rich.progress import Progress, SpinnerColumn, TimeElapsedColumn
from rich.table import Table
from sagemaker.deserializers import JSONDeserializer
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer

from utils.autoscaling import (
    monitor_scaling_events,
    print_scaling_times,
    test_concurrency_level,
)

from utils.llmperf import (
    print_llmperf_results,
    trigger_auto_scaling,
    monitor_process,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Initiate sagemaker session

In [6]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = sess.sagemaker_client
sagemaker_runtime_client = sess.sagemaker_runtime_client
cloudwatch_client = boto3.client("cloudwatch", region_name=region)

hf_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# retrieve the llm image uri
# tgi_dlc = f"763104351884.dkr.ecr.{region}.amazonaws.com/huggingface-pytorch-tgi-inference:2.1-tgi2.0-gpu-py310-cu121-ubuntu22.04"
tgi_dlc = get_huggingface_llm_image_uri("huggingface", version="2.0.0")

print(f"TGI DLC: \n[b i green]{tgi_dlc}")
print(f"Region: [b blue]{region}")
print(f"Role: [b red]{role}")

TGI DLC: 
763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.1-tgi2.0.0-gpu-py310-cu121-ubunt
u22.04

Region: us-east-1

Role: arn:aws:iam::057716757052:role/gen_ai_gsmoon

## Deploy model

Create and deploy model using Amazon SageMaker HuggingFace TGI DLC

<https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy>

<div class="alert alert-block alert-warning">
<b>NOTE:</b> Remember to copy your Hugging Face Access Token from <a href="https://hf.co/">https://hf.co/</a> before running the below cell.<br/><br/>
Refer <a href="https://huggingface.co/docs/hub/security-tokens">here</a> to learn about creating HF tokens.
</div>

In [7]:
# sagemaker config
instance_type = "ml.g5.2xlarge"
suffix = f"{str(uuid4())[:5]}-{datetime.now().strftime('%d%b%Y')}"
model_name = f"Llama3-8B-fas-{suffix}"
endpoint_name = model_name
health_check_timeout = 900

HF_TOKEN = os.getenv("HUGGING_FACE_HUB_TOKEN") or getpass("Enter HUGGINGFACE Access Token: ")
# Define Model and Endpoint configuration parameter
config = {
    "HF_MODEL_ID": "meta-llama/Meta-Llama-3-8B-Instruct",  # model_id from hf.co/models
    "SM_NUM_GPUS": "1",  # Number of GPU used per replica
    "MAX_INPUT_LENGTH": "2048",  # Max length of input text
    "MAX_TOTAL_TOKENS": "4096",  # Max length of the generation (including input text)
    "MAX_BATCH_TOTAL_TOKENS": "8192",  # Limits the number of tokens that can be processed in parallel during the generation
    "MESSAGES_API_ENABLED": "true",  # Enable the messages API
    "HUGGING_FACE_HUB_TOKEN": HF_TOKEN,
}

# create HuggingFaceModel with the image uri
print(f"Creating model: [b green]{model_name}...")
llm_model = HuggingFaceModel(name=model_name, role=role, image_uri=tgi_dlc, env=config)

# Deploy model to Amazon SageMaker endpoint
print(f"Deploying model to endpoint: [b magenta]{endpoint_name}...")
predictor = llm_model.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=1,
    instance_type=instance_type,
    container_startup_health_check_timeout=health_check_timeout,  # 15 minutes to be able to load the model
)

Enter HUGGINGFACE Access Token:  ········


Creating model: Llama3-8B-fas-ed594-28Jul2024...

Deploying model to endpoint: Llama3-8B-fas-ed594-28Jul2024...

-------------!

## Inference

Invoke and test endpoint using messages API. Refer to HF [Messages API](https://huggingface.co/docs/text-generation-inference/messages_api) for more info.

In [8]:
# Prepare prompt in messages API format
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is deep learning?"},
]

# Generation arguments
parameters = {
    "model": hf_model_id,  # model id is required
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 512,
    "stop": ["<|eot_id|>"],
}

chat = predictor.predict({"messages": messages, **parameters})

# Unpack and print response
print(chat["choices"][0]["message"]["content"].strip())

Deep learning is a subset of machine learning that involves the use of artificial neural networks to model and 
analyze complex data. These neural networks are composed of multiple layers of interconnected nodes or "neurons," 
which process and transform the input data in a hierarchical manner.

In traditional machine learning, algorithms are designed to learn from data by identifying patterns and 
relationships between features. In contrast, deep learning algorithms are designed to learn from data by 
identifying hierarchical patterns and relationships between features. This is achieved by training the neural 
network on a large dataset, allowing it to learn the underlying structure and patterns in the data.

Deep learning has been incredibly successful in a wide range of applications, including:

1. Computer Vision: Deep learning has revolutionized the field of computer vision, enabling machines to recognize 
and classify objects, scenes, and activities with high accuracy.
2. Natural Language Processing: Deep learning has enabled machines to understand and generate human language, 
including speech recognition, language translation, and text summarization.
3. Speech Recognition: Deep learning has improved speech recognition accuracy, enabling machines to recognize 
spoken words and phrases with high accuracy.
4. Robotics: Deep learning has enabled robots to learn and adapt to new tasks and environments, improving their 
ability to interact with humans and other objects.
5. Healthcare: Deep learning has been used to analyze medical images, predict patient outcomes, and develop 
personalized treatment plans.

Some of the key characteristics of deep learning include:

1. Hierarchical representations: Deep learning models learn to represent data in a hierarchical manner, with early 
layers learning low-level features and later layers learning high-level abstractions.
2. Non-linear transformations: Deep learning models learn to perform non-linear transformations on the input data, 
allowing them to capture complex patterns and relationships.
3. Large datasets: Deep learning models require large datasets to learn from, as they need to be trained on a large
number of examples to learn the underlying patterns and relationships.
4. Computational power: Deep learning models require significant computational power to train, as they involve 
complex mathematical operations and large amounts of data.

Some of the popular deep learning frameworks and libraries include:

1. TensorFlow
2. PyTorch
3. Keras
4. Caffe
5. OpenCV

I hope this helps! Let me know if you have any further questions.

## Baseline average latency at various concurrency levels (Optional)

By capturing average latency across various concurrency levels, we can get a fair idea on after how many concurrent request does endpoint performance would degrade significantly.

Having this information can help define values for scaling policy accordingly.

<div class="alert alert-block alert-info">
<b>Running below cell is optional</b><br/><br/>
<b>INFO: ℹ️</b> Signal here is, at a given concurrency level you start to see average latency increase significantly.<br/>
At this concurrency level the endpoint gets overloaded and cannot serve requests in a timely fashion.<br/>
We use these values to set as threshold values for autoscaling.
<br/><br/>
<b>NOTE: ⚠️</b> As concurrent requests to the endpoint increase you might observe <b>ThrottlingException</b> errors as we haven't incorporated exponential backoff and retry mechanisms.
</div>

In [9]:
# Define list of prompts
prompts = [
    "what is deep learning?",
    "what are various inference modes in Amazon SageMaker?",
    "Can I host Large language models on Amazon SageMaker?",
    "Does Amazon SageMaker support TensorRT-LLM?",
    "what is step scaling policy in the context of autoscaling ec2 instances on AWS?",
    "Why is the sky blue?",
    "List 5 benefits of incorporating limes into the diet.",
]

# Test different concurrency levels and measure average latency
concurrency_levels = [10, 50, 75, 100]  # Adjust these values as needed

for concurrency_level in concurrency_levels:
    try:
        avg_latency = test_concurrency_level(
            concurrency_level,
            prompts,
            messages,
            parameters,
            endpoint_name,
            sagemaker_runtime_client,
        )
        print(
            f"[b]Concurrency:[/b] {concurrency_level} requests,"
            f" [b]Average latency:[/b] {avg_latency:.2f} seconds"
        )
    except Exception as e:
        print(f"[b]At Concurrency[/b] {concurrency_level} requests," f"[b]Exception:[/b] \n{e}")
        continue

Concurrency: 10 requests, Average latency: 12.99 seconds

Concurrency: 50 requests, Average latency: 18.89 seconds

Concurrency: 75 requests, Average latency: 22.91 seconds

Concurrency: 100 requests, Average latency: 28.07 seconds

---

## Apply Autoscaling policies to the endpoint

Apply Application Autoscaling Policy to endpoint

1. Register Scalable Target

In [10]:
variant_name = "AllTraffic"
as_min_capacity = 1
as_max_capacity = 2

resource_id = f"endpoint/{endpoint_name}/variant/{variant_name}"

autoscaling_client = boto3.client("application-autoscaling", region_name=region)

# Register scalable target
scalable_target = autoscaling_client.register_scalable_target(
    ServiceNamespace="sagemaker",
    ResourceId=resource_id,
    ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    MinCapacity=as_min_capacity,
    MaxCapacity=as_max_capacity,  # Replace with your desired maximum instances
)

scalable_target_arn = scalable_target["ScalableTargetARN"]
print(f"Resource ID: [b blue]{resource_id}")
print(f"Scalable_target_arn:\n[b green]{scalable_target_arn}")

Resource ID: endpoint/Llama3-8B-fas-ed594-28Jul2024/variant/AllTraffic

Scalable_target_arn:
arn:aws:application-autoscaling:us-east-1:057716757052:scalable-target/056m05c090ea97e24453aa7f6ec654913dcb

## Use the latest high-resolution Metrics to trigger auto-scaling

- New feature introduces a new <span style='color:green'><b>PredefinedMetricType</b></span> for scaling policy configuration i.e. <span style='color:green'><b>SageMakerVariantConcurrentRequestsPerModelHighResolution</b></span> to trigger scaling actions.
- Creating a scaling policy with this metric type will create cloudwatch alarms that track a new metric called <span style='color:green'><b>ConcurrentRequestsPerModel</b></span>.
- These high-resolution metrics are published at sub-minute intervals (10s intervals to CW + any additional jitter + delays)
- We should observe significant improvement in scale out times with this new metric


### Steps to create Application autoscaling policy

- Create scaling policy
  - Set `PolicyType` to `TargetTrackingScaling`
  - Set `TargetValue` to `5.0`. i.e., Scaling triggers when endpoint receives 5 `ConcurrentRequestsPerModel`
  - Set `PredefinedMetricType` to `SageMakerVariantConcurrentRequestsPerModelHighResolution`
  - Set `ScaleInCoolDown` and `ScaleOutCoolDown` values to `300` seconds

In [11]:
# Create Target Tracking Scaling Policy
target_tracking_policy_response = autoscaling_client.put_scaling_policy(
    PolicyName="SageMakerEndpointScalingPolicy",
    ServiceNamespace="sagemaker",
    ResourceId=resource_id,
    ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    PolicyType="TargetTrackingScaling",
    TargetTrackingScalingPolicyConfiguration={
        "TargetValue": 5.0,  # Scaling triggers when endpoint receives 5 ConcurrentRequestsPerModel
        "PredefinedMetricSpecification": {
            "PredefinedMetricType": "SageMakerVariantConcurrentRequestsPerModelHighResolution"
        },
        "ScaleInCooldown": 300,  # Cooldown period after scale-in activity
        "ScaleOutCooldown": 300,  # Cooldown period after scale-out activity
    },
)

# print(target_tracking_policy_response)
print(f"[b]Policy ARN:[/b] [i blue]{target_tracking_policy_response['PolicyARN']}")

# print Cloudwatch Alarms
alarms = target_tracking_policy_response["Alarms"]

for alarm in alarms:
    print(f"[b]Alarm Name:[/b] [b magenta]{alarm['AlarmName']}")
    # print(f"[b]Alarm ARN:[/b] [i green]{alarm['AlarmARN']}[/i green]")
    print("===" * 15)

Policy ARN: 
arn:aws:autoscaling:us-east-1:057716757052:scalingPolicy:05c090ea-97e2-4453-aa7f-6ec654913dcb:resource/sagemaker/en
dpoint/Llama3-8B-fas-ed594-28Jul2024/variant/AllTraffic:policyName/SageMakerEndpointScalingPolicy

Alarm Name: 
TargetTracking-endpoint/Llama3-8B-fas-ed594-28Jul2024/variant/AllTraffic-AlarmHigh-db616fd5-f691-4f2e-a97c-3fc9dcba
d4c1

=============================================

Alarm Name: 
TargetTracking-endpoint/Llama3-8B-fas-ed594-28Jul2024/variant/AllTraffic-AlarmLow-54caa634-1e27-4dca-b42b-831c694b7
9e2

=============================================

## Trigger autoscaling action

### LLMPerf to generate traffic to the endpoint

Refer to <https://github.com/philschmid/llmperf> for more details on LLMPerf.

Run the LLMPerf traffic generation script in the background using `subprocess.Popen`

<div class="alert alert-block alert-info">
<b>INFO:ℹ️</b> Refer to <a href="utils/llmperf.py"><b>utils/llmperf.py</b></a> for <span style='color:red'>trigger_autoscaling</span> function implementation
</div>

### Monitor Scale-Out Alarm Trigger times and scaling event times

As llmperf generates traffic to the endpoint continuously this trigger auto-scaling.

The `monitor_scaling_events` function does the following:
- Calculates time taken for alarm to go into InAlarm state.
- checks if alarm is InAlarm state. If yes, then starts the scaling timer
- continuously monitors the `DesiredInstanceCount` property of the endpoint
  - waits till `CurrentInstanceCount == DesiredInstanceCount` and `EndpointStatus` is `InService`
- Calculates time taken to scale out instances prints the times in a table

The below cell triggers auto scaling action and calls the monitor_scaling_events immediately on the AlarmHigh

<div class="alert alert-block alert-info">
<b>INFO: ℹ️</b> Refer to <a href="utils/autoscaling.py"><b>utils/autoscaling.py</b></a> for <span style='color:red'>monitor_scaling_events</span> function implementation
</div>

<div class="alert alert-block alert-info">
<b>NOTE: ⚠️</b>The <b>AlarmHigh</b> Alarm triggers scale out actions only after the threshold of <b>ConcurrentRequestsPerModel >5 </b> for 3 datapoints within <b>30 seconds</b> is breached.
</div>

In [12]:
# Trigger LLMPerf script to generate traffic to endpoint
num_concurrent_requests = 100
# LLMperf requires session credentials be passed in via environment variables.
# We'll use the current session to get these credentials.
creds = boto_session.get_credentials()
process = trigger_auto_scaling(creds, region, endpoint_name, num_concurrent_requests)
print(f"[b green]Process ID for LLMPerf: {process.pid}")

# get AlarmHigh alarm name
scaleout_alarm_name = [alarm["AlarmName"] for alarm in alarms if "AlarmHigh" in alarm["AlarmName"]][
    0
]

# Start monitoring scaling events
SLEEP_TIME = 5  # time to sleep
scaling_times = monitor_scaling_events(
    endpoint_name, scaleout_alarm_name, SLEEP_TIME, cloudwatch_client, sagemaker_client
)

# Print scaling times
console = Console()
table = print_scaling_times(scaling_times)
console.print(table)

Calling LLMPerf shell script: 
/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/30_fine_tune/03-fine-tune-llama3/notebook/02-nav
er-news-llama3-inference/backup/01-Llama3-8B-Autoscaling/trigger_autoscaling.sh

Launching LLMPerf with 100 concurrent requests

Process ID for LLMPerf: 19315

Initial instance count: 1

Tracking Alarm: 
TargetTracking-endpoint/Llama3-8B-fas-ed594-28Jul2024/variant/AllTraffic-AlarmHigh-db616fd5-f691-4f2e-a97c-3fc9dcba
d4c1

Output()

                  Scaling Times                   
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Target Instance Count ┃ Scaling Time (seconds) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│                     2 │                 433.01 │
└───────────────────────┴────────────────────────┘

### Monitor if the background process (llmperf) is completed.

In [13]:
monitor_process(process)

Process 19315 finished with return code 0

Process output:
Installing llmperf...
results directory already exists.
Starting benchmarking scripts on endpoint Llama3-8B-fas-ed594-28Jul2024 ...
[33m(raylet)[0m WARNING: 208 PYTHON worker processes have been started on node: 
b0fb231c1c2f493c5c253059300ca732c5ef88c1c915c1504ed05ba0 with address: 172.16.34.227. This could be a result of 
using a large number of actors, or due to tasks blocked in ray.get() calls (see 
https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
\Results for token benchmark for Llama3-8B-fas-ed594-28Jul2024 queried with the sagemaker api.

inter_token_latency_s
    p25 = 0.020720480832449085
    p50 = 0.028100433350352054
    p75 = 0.03507947415856997
    p90 = 0.04170722566765609
    p95 = 0.0466553749129518
    p99 = 0.05846200418841567
    mean = 0.028477163436279616
    min = 0.0011077229858391694
    max = 0.11532974299970428
    stddev = 0.011396205458922745
ttft_s
    p25 = 4.145546742423903
    p50 = 9.207395828445442
    p75 = 14.604626631247811
    p90 = 15.029941626801156
    p95 = 15.04388886326924
    p99 = 15.053662227115128
    mean = 8.834211754302494
    min = 0.10703785298392177
    max = 15.078184877987951
    stddev = 5.19258135747693
end_to_end_latency_s
    p25 = 14.056195328885224
    p50 = 19.48325969092548
    p75 = 24.505333405570127
    p90 = 27.500939839775675
    p95 = 28.905332016572356
    p99 = 31.480270544991363
    mean = 19.015451940153028
    min = 0.7931988791096956
    max = 35.920733163831756
    stddev = 6.865291040218732
request_output_throughput_token_per_s
    p25 = 28.50654190883261
    p50 = 35.5854345939724
    p75 = 48.26065785675587
    p90 = 69.53173406291391
    p95 = 86.30822417919369
    p99 = 189.69233374101057
    mean = 44.460940950254326
    min = 8.67064585668825
    max = 902.6739936945633
    stddev = 41.80154524963839
number_input_tokens
    p25 = 441.5
    p50 = 550.5
    p75 = 652.0
    p90 = 736.0
    p95 = 791.0999999999999
    p99 = 889.1099999999999
    mean = 549.734
    min = 108
    max = 1066
    stddev = 146.00740188567636
number_output_tokens
    p25 = 578.0
    p50 = 690.0
    p75 = 797.0
    p90 = 908.0
    p95 = 965.1999999999998
    p99 = 1088.02
    mean = 689.465
    min = 216
    max = 1212
    stddev = 166.34207664166473
Number Of Errored Requests: 0
Overall Output Throughput: 1643.4463844862348
Number Of Completed Requests: 1000
Completed Requests Per Minute: 143.01927301483627
[33m(raylet)[0m WARNING: 282 PYTHON worker processes have been started on node: 
b0fb231c1c2f493c5c253059300ca732c5ef88c1c915c1504ed05ba0 with address: 172.16.34.227. This could be a result of 
using a large number of actors, or due to tasks blocked in ray.get() calls (see 
https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
Execution time was 430 secs.

Process errors:
Cloning into 'llmperf'...
[2mResolved [1m104 packages[0m in 147ms[0m
[2mDownloaded [1m12 packages[0m in 676ms[0m
[2mUninstalled [1m4 packages[0m in 79ms[0m
[2mInstalled [1m47 packages[0m in 56ms[0m
 [32m+[39m [1maiohttp[0m[2m==3.9.5[0m
 [32m+[39m [1maiosignal[0m[2m==1.3.1[0m
 [32m+[39m [1masync-timeout[0m[2m==4.0.3[0m
 [31m-[39m [1mboto3[0m[2m==1.34.142[0m
 [32m+[39m [1mboto3[0m[2m==1.34.101[0m
 [31m-[39m [1mbotocore[0m[2m==1.34.142[0m
 [32m+[39m [1mbotocore[0m[2m==1.34.101[0m
 [32m+[39m [1mcachetools[0m[2m==5.4.0[0m
 [32m+[39m [1mdistro[0m[2m==1.9.0[0m
 [32m+[39m [1mdocopt[0m[2m==0.6.2[0m
 [32m+[39m [1mdocstring-parser[0m[2m==0.16[0m
 [32m+[39m [1mfrozenlist[0m[2m==1.4.1[0m
 [32m+[39m [1mgoogle-api-core[0m[2m==2.19.1[0m
 [32m+[39m [1mgoogle-auth[0m[2m==2.32.0[0m
 [32m+[39m [1mgoogle-cloud-aiplatform[0m[2m==1.60.0[0m
 [32m+[39m [1mgoogle-cloud-bigquery[0m[2m==3.25.0[0m
 [32m+[39m [1mgoogle-cloud-core[0m[2m==2.4.1[0m
 [32m+[39m [1mgoogle-cloud-resource-manager[0m[2m==1.12.4[0m
 [32m+[39m [1mgoogle-cloud-storage[0m[2m==2.18.0[0m
 [32m+[39m [1mgoogle-crc32c[0m[2m==1.5.0[0m
 [32m+[39m [1mgoogle-resumable-media[0m[2m==2.7.1[0m
 [32m+[39m [1mgoogleapis-common-protos[0m[2m==1.63.2[0m
 [32m+[39m [1mgrpc-google-iam-v1[0m[2m==0.13.1[0m
 [32m+[39m [1mgrpcio[0m[2m==1.65.1[0m
 [32m+[39m [1mgrpcio-status[0m[2m==1.62.2[0m
 [32m+[39m [1mhuggingface-hub[0m[2m==0.24.2[0m
 [32m+[39m [1miniconfig[0m[2m==2.0.0[0m
 [32m+[39m [1mlitellm[0m[2m==1.42.5[0m
 [32m+[39m [1mllmperf[0m[2m==0.1.0 (from 
file:///home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/30_fine_tune/03-fine-tune-llama3/notebook
/02-naver-news-llama3-inference/backup/01-Llama3-8B-Autoscaling/llmperf)[0m
 [32m+[39m [1mmsgpack[0m[2m==1.0.8[0m
 [32m+[39m [1mmultidict[0m[2m==6.0.5[0m
 [32m+[39m [1mnum2words[0m[2m==0.5.13[0m
 [32m+[39m [1mopenai[0m[2m==1.37.1[0m
 [32m+[39m [1mpluggy[0m[2m==1.5.0[0m
 [32m+[39m [1mproto-plus[0m[2m==1.24.0[0m
 [32m+[39m [1mpyasn1-modules[0m[2m==0.4.0[0m
 [31m-[39m [1mpydantic[0m[2m==2.6.4 (from 
file:///home/conda/feedstock_root/build_artifacts/pydantic_1710622263606/work)[0m
 [32m+[39m [1mpydantic[0m[2m==2.4.2[0m
 [31m-[39m [1mpydantic-core[0m[2m==2.16.3 (from 
file:///home/conda/feedstock_root/build_artifacts/pydantic-core_1708700740324/work)[0m
 [32m+[39m [1mpydantic-core[0m[2m==2.10.1[0m
 [32m+[39m [1mpytest[0m[2m==8.3.2[0m
 [32m+[39m [1mray[0m[2m==2.33.0[0m
 [32m+[39m [1mregex[0m[2m==2024.7.24[0m
 [32m+[39m [1msafetensors[0m[2m==0.4.3[0m
 [32m+[39m [1mshapely[0m[2m==2.0.5[0m
 [32m+[39m [1mshellingham[0m[2m==1.5.4[0m
 [32m+[39m [1mtiktoken[0m[2m==0.7.0[0m
 [32m+[39m [1mtokenizers[0m[2m==0.19.1[0m
 [32m+[39m [1mtransformers[0m[2m==4.43.3[0m
 [32m+[39m [1mtyper[0m[2m==0.12.3[0m
 [32m+[39m [1myarl[0m[2m==1.9.4[0m
2024-07-28 08:40:41,610 INFO worker.py:1781 -- Started a local Ray instance.
You are using the default legacy behaviour of the <class 
'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that 
the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set
`legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this 
was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer 
from a GGUF file you can ignore this message.
  0%|          | 0/1000 [00:00<?, ?it/s]  0%|          | 0/1000 [00:00<?, ?it/s]  0%|          | 0/1000 [00:00<?, 
?it/s]  0%|          | 0/1000 [00:00<?, ?it/s]  0%|          | 0/1000 [00:00<?, ?it/s]  0%|          | 0/1000 
[00:00<?, ?it/s]  0%|          | 0/1000 [00:01<?,

## Print LLMPerf results

LLMPerf writes the results to **"results/"** directory.  `summary.json` file has the endpoint benchmarking data.

In [14]:
print_llmperf_results(num_concurrent_requests)

            LLMPerf Endpoint Metrics             
                           ╷                     
                    Metric │ Units               
 ══════════════════════════╪════════════════════ 
       Concurrent requests │ 100                 
   Avg. Input token length │ 550                 
  Avg. Output token length │ 150                 
  Avg. First-Time-To-Token │ 8834.21ms           
           Avg. Thorughput │ 1643.45 tokens/sec  
              Avg. Latency │ 28.48ms/token       
                           ╵

### Monitor Scale-in Alarm Trigger times and scaling event times

<div class="alert alert-block alert-warning">
<b>NOTE: ⚠️</b>The <b>AlarmLow</b> Alarm triggers scale-in actions only after the threshold of <b>ConcurrentRequestsPerModel < 4.5</b> for 90 datapoints within <b>15 minutes</b> is breached.
<br/>Running the below cell with take approximately 15 minutes to complete.<br/>
</div>

In [15]:
# get AlarmHigh alarm name
scalein_alarm_name = [alarm["AlarmName"] for alarm in alarms if "AlarmLow" in alarm["AlarmName"]][0]

# Start monitoring scaling events
SLEEP_TIME = 5  # time to sleep
scaling_times = monitor_scaling_events(
    endpoint_name, scalein_alarm_name, SLEEP_TIME, cloudwatch_client, sagemaker_client
)

# Print scaling times
console = Console()
table = print_scaling_times(scaling_times)
console.print(table)

Initial instance count: 2

Tracking Alarm: 
TargetTracking-endpoint/Llama3-8B-fas-ed594-28Jul2024/variant/AllTraffic-AlarmLow-54caa634-1e27-4dca-b42b-831c694b7
9e2

Output()

                  Scaling Times                   
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Target Instance Count ┃ Scaling Time (seconds) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│                     1 │                  86.07 │
└───────────────────────┴────────────────────────┘

## Cleanup

- Deregister scalable target. This automatically deletes associated cloudwatch alarms.
- Delete model
- Delete endpoint

In [16]:
# Deregister the scalable target for AAS
try:
    autoscaling_client.deregister_scalable_target(
        ServiceNamespace="sagemaker",
        ResourceId=resource_id,
        ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    )
    print(f"Scalable target for [b]{resource_id}[/b] deregistered. ✅")
except autoscaling_client.exceptions.ObjectNotFoundException:
    print(f"Scalable target for [b]{resource_id}[/b] not found!.")

print("---" * 10)
# Delete model and endpoint
try:
    print(f"Deleting model: [b green]{model_name} ✅")
    predictor.delete_model()
except Exception as e:
    print(f"{e}")

try:
    print(f"Deleting endpoint: [b magenta]{predictor.endpoint_name} ✅")
    predictor.delete_endpoint()
except Exception as e:
    print(f"{e}")

print("---" * 10)
print(f"Done")

Scalable target for endpoint/Llama3-8B-fas-ed594-28Jul2024/variant/AllTraffic deregistered. ✅

------------------------------

Deleting model: Llama3-8B-fas-ed594-28Jul2024 ✅

Deleting endpoint: Llama3-8B-fas-ed594-28Jul2024 ✅

------------------------------

Done

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb)